In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# Constants
IMG_SIZE = (128, 128)  # Resize images to 128x128 pixels

In [ ]:
# Directories
TRAINING_DATA_DIR = "/content/drive/MyDrive/SkinCancerDataset/ISBI2016_ISIC_Part1_Training_Data"  # Modify if your directory name is different
TRAINING_GT_DIR = "/content/drive/MyDrive/SkinCancerDataset/ISBI2016_ISIC_Part1_Training_GroundTruth"  # Modify if your directory name is different

In [ ]:
# 1. Data Loading and Preprocessing
def preprocess_image(image_path):
    """Load, resize, and normalize the image."""
    img = Image.open(image_path)
    img_resized = img.resize(IMG_SIZE)
    img_array = np.array(img_resized) / 255.0
    return img_array

X_train = []
Y_train = []

for img_file in os.listdir(TRAINING_DATA_DIR):
    img_path = os.path.join(TRAINING_DATA_DIR, img_file)
    X_train.append(preprocess_image(img_path))

    # Load segmentation mask
    seg_file = img_file.replace(".jpg", "_Segmentation.png")
    seg_path = os.path.join(TRAINING_GT_DIR, seg_file)
    if os.path.exists(seg_path):
        Y_train.append(preprocess_image(seg_path))
    else:
        blank_img = Image.new('RGB', IMG_SIZE)
        Y_train.append(np.array(blank_img) / 255.0)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
# 2. Splitting the data
X_train_final, X_val, Y_train_final, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)


In [ ]:
# 3. Data Augmentation (the new part)
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()  # We don't augment validation data

# Create Python generators
batch_size = 16
train_generator = train_datagen.flow(X_train_final, Y_train_final, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, Y_val, batch_size=batch_size)

In [ ]:
# 4. Model Building
def build_model(input_shape):
    """Define the enhanced U-Net model."""

    # Input layer
    inputs = Input(input_shape)

    # Contracting path
    c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.1)(c3)
    c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)

    # Expanding path
    u3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c4)
    u3 = concatenate([u3, c3])
    c5 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u3)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    u2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c5)
    u2 = concatenate([u2, c2])
    c6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u2)
    c6 = Dropout(0.1)(c6)
    c6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u1 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c6)
    u1 = concatenate([u1, c1])
    c7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u1)
    c7 = Dropout(0.1)(c7)
    c7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    # Output layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [ ]:
input_shape = (128, 128, 3)
model = build_model(input_shape)


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# [Set up Early Stopping and Model Checkpointing]
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
callbacks_list = [early_stopping, checkpoint]

In [ ]:
# 5. Compile and Train the Model
history = model.fit(train_generator, validation_data=val_generator, epochs=50, steps_per_epoch=len(X_train_final) // 32, callbacks=callbacks_list)

Epoch 1/50
22/22 [==============================] - ETA: 0s - loss: 0.5708 - accuracy: 0.7210
Epoch 1: val_loss improved from inf to 0.51927, saving model to best_model.h5
22/22 [==============================] - 22s 155ms/step - loss: 0.5708 - accuracy: 0.7210 - val_loss: 0.5193 - val_accuracy: 0.6981
Epoch 2/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


22/22 [==============================] - ETA: 0s - loss: 0.5163 - accuracy: 0.7551
Epoch 2: val_loss improved from 0.51927 to 0.41710, saving model to best_model.h5
22/22 [==============================] - 4s 194ms/step - loss: 0.5163 - accuracy: 0.7551 - val_loss: 0.4171 - val_accuracy: 0.8273
Epoch 3/50
22/22 [==============================] - ETA: 0s - loss: 0.4811 - accuracy: 0.7880
Epoch 3: val_loss did not improve from 0.41710
22/22 [==============================] - 2s 108ms/step - loss: 0.4811 - accuracy: 0.7880 - val_loss: 0.4375 - val_accuracy: 0.8194
Epoch 4/50
22/22 [==============================] - ETA: 0s - loss: 0.4922 - accuracy: 0.7843
Epoch 4: val_loss did not improve from 0.41710
22/22 [==============================] - 2s 106ms/step - loss: 0.4922 - accuracy: 0.7843 - val_loss: 0.4427 - val_accuracy: 0.8156
Epoch 5/50
22/22 [==============================] - ETA: 0s - loss: 0.4777 - accuracy: 0.7764
Epoch 5: val_loss improved from 0.41710 to 0.40412, saving model t

In [ ]:
# 6. Save the model
model.save("skin_cancer_segmentation_model.h5")

In [ ]:
# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(X_val, Y_val)

print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")


6/6 [==============================] - 3s 260ms/step - loss: 0.2379 - accuracy: 0.8949
Validation Loss: 0.23786218464374542
Validation Accuracy: 0.8948940634727478


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import numpy as np

# Load the trained U-Net model
unet_model = load_model('unet_best_model.h5')

# Test dataset directory
test_images_dir = '/content/drive/MyDrive/SkinCancerDataset/ISBI2016_ISIC_Part1_Test_Data/'

In [ ]:
# Load and preprocess the test images
test_image_files = os.listdir(test_images_dir)
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

X_test = np.zeros((len(test_image_files), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
for n, image_file in enumerate(test_image_files):
    image_path = os.path.join(test_images_dir, image_file)
    img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    X_test[n] = img_to_array(img)

In [ ]:
# Predict using the U-Net model
unet_predictions = unet_model.predict(X_test)

12/12 [==============================] - 11s 138ms/step


In [ ]:
# Save the predictions as images with appropriate naming convention
save_dir = '/content/drive/MyDrive/SkinCancerDataset/predicted_masks_unet/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
from PIL import Image

for i, pred_mask in enumerate(unet_predictions):
    # Squeeze out the last dimension to get a 2D array
    mask_2d = np.squeeze(pred_mask, axis=-1)

    # Convert the mask values to the range [0, 255] and then to uint8 type
    mask_to_save = (mask_2d * 255).astype(np.uint8)

    # Convert the numpy array to a PIL image
    mask_img = Image.fromarray(mask_to_save)

    # Extract the base name of the original test image
    base_name = os.path.splitext(test_image_files[i])[0]

    # Create the filename for the predicted mask
    mask_filename = os.path.join(save_dir, base_name + "_Segmentation.png")

    # Save the PIL image
    mask_img.save(mask_filename)


In [ ]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize

# Paths for the predicted masks and ground truth masks
predicted_masks_path = '/content/drive/MyDrive/SkinCancerDataset/predicted_masks_unet/'
ground_truth_path = '/content/drive/MyDrive/SkinCancerDataset/ISBI2016_ISIC_Part1_Test_GroundTruth/'

def compute_iou(mask1, mask2):
    """Compute Intersection over Union (IoU) of two binary masks."""
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    return np.sum(intersection) / np.sum(union)

def compute_dice(mask1, mask2):
    """Compute Dice Coefficient of two binary masks."""
    intersection = np.logical_and(mask1, mask2)
    return 2. * np.sum(intersection) / (np.sum(mask1) + np.sum(mask2))

ious = []
dices = []

for mask_file in os.listdir(predicted_masks_path):
    predicted_mask = imread(os.path.join(predicted_masks_path, mask_file))
    ground_truth_mask_original = imread(os.path.join(ground_truth_path, mask_file))

    # Resize the ground truth mask to 128x128
    ground_truth_mask = resize(ground_truth_mask_original, (128, 128), mode='constant', preserve_range=True)

    # Binarize the masks for metric computation
    predicted_mask_binarized = (predicted_mask > 127).astype(np.uint8)
    ground_truth_mask_binarized = (ground_truth_mask > 127).astype(np.uint8)

    iou = compute_iou(predicted_mask_binarized, ground_truth_mask_binarized)
    dice = compute_dice(predicted_mask_binarized, ground_truth_mask_binarized)

    ious.append(iou)
    dices.append(dice)

# Compute average IoU and Dice Coefficient
avg_iou = np.mean(ious)
avg_dice = np.mean(dices)

print(f"Average IoU: {avg_iou:.4f}")
print(f"Average Dice Coefficient: {avg_dice:.4f}")


Average IoU: 0.5725
Average Dice Coefficient: 0.6818


In [ ]:
import os
import numpy as np
from skimage.io import imread

# Directory paths
predicted_masks_path = '/content/drive/MyDrive/SkinCancerDataset/predicted_masks_unet/'

# Get list of predicted mask files
mask_files = os.listdir(predicted_masks_path)

# Lists to store results
coverage_percentages = []
risk_categories = []
interpretations = []

# Iterate through each predicted mask
for mask_file in mask_files:
    mask_path = os.path.join(predicted_masks_path, mask_file)
    mask = imread(mask_path)

    # Binarize the mask
    mask_binarized = (mask > 127).astype(np.uint8)

    # Compute coverage percentage
    coverage = (np.sum(mask_binarized) / (mask_binarized.shape[0] * mask_binarized.shape[1])) * 100
    coverage_percentages.append(coverage)

    # Determine risk category and interpretation
    if coverage < 5:
        risk_categories.append("Low Risk")
        interpretations.append("The analysis indicates a low presence of abnormal tissue. While it appears likely benign, always consider a professional dermatologist's advice.")
    elif coverage >= 5 and coverage <= 15:
        risk_categories.append("Medium Risk")
        interpretations.append("The model has identified some regions of potential concern. It's recommended to consult a dermatologist for a more detailed examination.")
    else:
        risk_categories.append("High Risk")
        interpretations.append("The analysis indicates a significant presence of abnormal tissue. Immediate consultation with a dermatologist is advised.")

# You can now use the lists `coverage_percentages`, `risk_categories`, and `interpretations` for further analysis or display.


In [ ]:
# Assuming you've run the previous code and have the lists `coverage_percentages`, `risk_categories`, and `interpretations`

for i, mask_file in enumerate(mask_files):
    print(f"Image: {mask_file}")
    print(f"Coverage Percentage: {coverage_percentages[i]:.2f}%")
    print(f"Risk Category: {risk_categories[i]}")
    print(f"Interpretation: {interpretations[i]}")
    print("-" * 50)  # to separate the results for each image


Image: ISIC_0000053_Segmentation.png
Coverage Percentage: 32.75%
Risk Category: High Risk
Interpretation: The analysis indicates a significant presence of abnormal tissue. Immediate consultation with a dermatologist is advised.
--------------------------------------------------
Image: ISIC_0000012_Segmentation.png
Coverage Percentage: 4.92%
Risk Category: Low Risk
Interpretation: The analysis indicates a low presence of abnormal tissue. While it appears likely benign, always consider a professional dermatologist's advice.
--------------------------------------------------
Image: ISIC_0000022_Segmentation.png
Coverage Percentage: 26.18%
Risk Category: High Risk
Interpretation: The analysis indicates a significant presence of abnormal tissue. Immediate consultation with a dermatologist is advised.
--------------------------------------------------
Image: ISIC_0000027_Segmentation.png
Coverage Percentage: 13.81%
Risk Category: Medium Risk
Interpretation: The model has identified some regi